In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from esios import *

Create dictionary with Indicator Name and Number of Indicator

In [2]:
indicatorsDict = {
    'Previsión diaria de la demanda eléctrica peninsular' : 460,
#                   'Generación prevista Solar' : 10034,
#                   'Previsión de la producción eólica nacional peninsular' : 541,  
                  'Precio medio horario componente mercado diario' : 805,
                  'Generación T.Real nuclear' : 549,
#                   'Generación T.Real Solar' : 10206,
                  'Generación T.Real hidráulica' : 546,
                  'Generación T.Real eólica' : 551,
                  'Generación T.Real Solar térmica' : 1294,
                  'Generación T.Real Solar fotovoltaica' : 1295
                 }

indicatorsItems = indicatorsDict.items()   # To iterate over name and number at the same time

Download indicators data from 01/01/2016 to 01/06/2020 and save it into `".csv"` files.

In [3]:
if __name__ == '__main__':
    start_date = "2016-01-01"
    end_date = "2020-05-01"
    start_ = start_date + 'T00:00:00'
    end_ = end_date + 'T23:50:00'
    token = '6cc21e0b60e9931e7522a6ce72a1a09f3a6fadc6f08b142f956db142c6858bc2'    # Introduce ESIOS token
    esios = ESIOS(token)
    for indicatorName, indicatorValue in indicatorsItems:
        print ('Start Date: ' + start_date)
        print ('End Date: ' + end_date)
        indicators_ = list()
        indicators_.append(indicatorValue)
        dfmul , df_list, names = esios.get_multiple_series_Spain(indicators_, start_, end_)
        df = dfmul[names]
        df = df.reset_index()
        df.columns = ['Date', indicatorName]
        df['Date'] = df['Date'].str.replace('.', ' ')
        df['Date'] = df['Date'].str.split().str[0]
        df['Date'] = df['Date'].str.replace('T', ' ')
        # Export to .csv file
        try:
            os.stat("Files/")
        except:
            os.mkdir("Files/")
        df.to_csv(path_or_buf= 'Files/' + str(indicatorValue) + '.csv', sep='^', index=False)
        print('Generated:' + str(indicatorValue))

Analyzing indicators...
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:460
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:805
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:549
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:546
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:551
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:1294
Start Date: 2019-01-01
End Date: 2019-05-01
Generated:1295


Read `'.csv'` files generated in different DataFrames.

In [ ]:
with open('Files/546.csv', 'r') as fichero:     
    dfDemand = pd.read_csv(fichero, sep = '^', dtype='object')
    dfDemand['Fecha']=pd.to_datetime(dfDemand['Fecha'])
    
with open('Files/10034.csv', 'r') as fichero:     
    dfSolar = pd.read_csv(fichero, sep = '^', dtype='object')
    dfSolar['Fecha']=pd.to_datetime(dfSolar['Fecha'])
    
with open('Files/541.csv', 'r') as fichero:     
    dfEolic = pd.read_csv(fichero, sep = '^', dtype='object')
    dfEolic['Fecha']=pd.to_datetime(dfEolic['Fecha'])
    
with open('Files/805.csv', 'r') as fichero:     
    dfPrice = pd.read_csv(fichero, sep = '^', dtype='object')
    dfPrice['Fecha']=pd.to_datetime(dfPrice['Fecha']) 

In [ ]:
dfDemand.sample(5)

In [ ]:
dfSolar.sample(5)

In [ ]:
dfEolic.sample(5)

In [ ]:
dfPrice.sample(5)